In [ ]:
!git clone https://github.com/acmarkes/traffic-sign-recog.git
%cd /content/traffic-sign-recog
!pip install gdown

import gdown
imgs = 'https://drive.google.com/uc?id=17an3cKGO1I2WlBPSa6zcAucL-jISSHEn'
labels = 'https://drive.google.com/uc?id=17gFHTQvU0utSNm3XMv2I8Ud4IzuqDLTl'

gdown.download(imgs, 'procTrainImages.joblib', quiet=False)
gdown.download(labels, 'train_labels.joblib', quiet=False)


In [ ]:
import joblib
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime as dt

import tensorflow as tf
from tensorflow import keras

import utils
from resnet import resnet18, resnet34, resnet50

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
procTrainImages = joblib.load('procTrainImages.joblib')
print(procTrainImages.shape)
utils.sample_images(procTrainImages, seed_num=42)
trainLabels = joblib.load('train_labels.joblib')
print(len(trainLabels))

In [ ]:
img_tweaks = {
    'featurewise_center':True,
    'featurewise_std_normalization':True,
    'rotation_range':20,
    'width_shift_range':0.2,
    'height_shift_range':0.2,
}

In [ ]:
Xy_train, Xy_val = utils.get_dataset_partitions_tf(procTrainImages, trainLabels, train_batch_size = 64, val_batch_size = 32, gen_kws=img_tweaks)

In [ ]:
model18 = resnet18(43)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) #early stopping to avoid overfitting and wasting time
filepath = 'best_resnet18_weights.hdf5'
checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')    #keeping saved weights of best result only
csv_logger = keras.callbacks.CSVLogger(f'log {dt.now().strftime("%Y-%m-%d %H:%M")} resnet18.csv', append=True, separator=';')

optimizer = keras.optimizers.Adam(learning_rate = 0.001)
model18.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history18 = model18.fit(Xy_train ,epochs=30, validation_data=Xy_val, callbacks=[early_stop, checkpoint, csv_logger])


In [ ]:
model34 = resnet34(43)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) #early stopping to avoid overfitting and wasting time
filepath = 'best_resnet34_weights.hdf5'
checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')    #keeping saved weights of best result only
csv_logger = keras.callbacks.CSVLogger(f'log {dt.now().strftime("%Y-%m-%d %H:%M")} resnet34.csv', append=True, separator=';')

optimizer = keras.optimizers.Adam(learning_rate = 0.001)
model34.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history34 = model34.fit(Xy_train ,epochs=30, validation_data=Xy_val, callbacks=[early_stop, checkpoint, csv_logger])


In [ ]:
model50 = resnet50(43)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) #early stopping to avoid overfitting and wasting time
filepath = 'best_resnet50_weights.hdf5'
checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')    #keeping saved weights of best result only
csv_logger = keras.callbacks.CSVLogger(f'log {dt.now().strftime("%Y-%m-%d %H:%M")} resnet50.csv', append=True, separator=';')

optimizer = keras.optimizers.Adam(learning_rate = 0.001)
model50.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history50 = model50.fit(Xy_train, epochs=30, validation_data=Xy_val, callbacks=[early_stop, checkpoint, csv_logger])


In [ ]:
import pandas as pd

pd.DataFrame(history18.history).plot(title= 'ResNet18' , figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.savefig('resnet18 performance.png')
plt.show()

In [ ]:
pd.DataFrame(history34.history).plot(title= 'ResNet34' , figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.savefig('resnet34 performance.png')
plt.show()

In [ ]:
pd.DataFrame(history50.history).plot(title='ResNet50', figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.savefig('resnet50 performance.png')
plt.show()

In [ ]:
tf.keras.utils.plot_model(model18, to_file='resnet18 architecture.png', show_shapes=True)
tf.keras.utils.plot_model(model34, to_file='resnet34 architecture.png', show_shapes=True)
tf.keras.utils.plot_model(model50, to_file='resnet50 architecture.png', show_shapes=True)